<a href="https://colab.research.google.com/github/alirezash97/BraTS/blob/master/Survival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
import tensorflow as tf
import os
import numpy as np
from nibabel.testing import data_path
import nibabel as nib
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import cv2
import keras 
import random
from keras import Model
from keras.optimizers import Adam

In [2]:
from sklearn.datasets import load_boston
from keras.models import Sequential
from keras.initializers import glorot_uniform
from keras.layers import Dense, Conv2D, Flatten, Input, BatchNormalization, Activation, MaxPooling2D, Add, AveragePooling2D, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [3]:
import glob, os
images_path = glob.glob('/content/drive/My Drive/BRATS2020/MICCAI_BraTS2020_TrainingData/**/*.nii.gz', recursive=True)

In [22]:

X_trainset_filenames = []

for item in images_path:
  if 'seg' in item:
    X_trainset_filenames.append(os.path.join(data_path, item))
  else:
    pass

In [23]:
print(len(X_trainset_filenames))

369


In [24]:
def shuffle(input_path):

  a_list = list(range(0, len(input_path)))
  random.shuffle(a_list)


  new_input_path = []
  for i in a_list:
    
    new_input_path.append(input_path[i])

  return new_input_path

In [25]:
 X_trainset_filenames = shuffle(X_trainset_filenames)

In [7]:
print( X_trainset_filenames[0])

/content/drive/My Drive/BRATS2020/MICCAI_BraTS2020_TrainingData/BraTS20_Training_242/BraTS20_Training_242_seg.nii.gz


In [ ]:
X_trainset_filenames[60][81:84]

'363'

In [8]:

def load_case(label_nifty_file):
    
    # load the label file, get the image content and return a numpy array for each

    label = np.array(nib.load(label_nifty_file).get_fdata())

    for i in range(240):
          for j in range(240):
            for k in range(155):
              if label[i, j, k] == 4.0 :
                label[i, j, k] = 3.0
              else:
                pass
    
    
    return label

In [10]:


from ipywidgets import interact

M = load_case(X_trainset_filenames[0])

def explore_3dimage(layer):
    plt.figure(figsize=(10, 5))
    channel = 3
    plt.imshow(M[:, :, layer]);
    plt.title('Explore Layers of Tumor prediction', fontsize=20)
    plt.axis('off')
    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_3dimage, layer=(0, 155 - 1));

interactive(children=(IntSlider(value=77, description='layer', max=154), Output()), _dom_classes=('widget-inte…

In [9]:

## get the most important part for survival task (the tumor)


def get_max(image, out_x, out_y, out_z, stride, number_of_images_to_extract):
  
  x = image.shape[0]
  y = image.shape[1]
  z = image.shape[2]


  total_number_of_cubes = ( int((z - out_z)/stride) * int((x - out_x)/stride) * int((y - out_y)/stride) )
  details = np.zeros((total_number_of_cubes, 4))

  counter = 0
  for i in range( int((z - out_z)/stride) ):
    for j in range( int((x - out_x)/stride) ):
      for k in range( int((y - out_y)/stride) ):

        
        temp = np.sum(image[ (j*stride):((j*stride)+out_x), (k*stride):((k*stride)+out_y), (i*stride):((i*stride)+out_z) ])
        details[counter] = (j*stride), (k*stride), (i*stride), temp
        counter += 1

  

  
  final_extracted_images = []
  for l in range(number_of_images_to_extract):
    
    max_ind = np.unravel_index(np.argmax(details, axis=None), details.shape) 
  
    x_start = int(details[max_ind[0], 0])
    y_start = int(details[max_ind[0], 1])
    z_start = int(details[max_ind[0], 2])

    temp = image[ x_start:(x_start+out_x) , y_start:(y_start+out_y) , z_start:(z_start+out_z) ]
    final_extracted_images.append(temp)
    details[max_ind[0], 3] = 0



  return np.array(final_extracted_images)




In [10]:
image = get_max(M, 64, 64, 64, 8, 3)

NameError: ignored

In [ ]:
print(image.shape)

(3, 64, 64, 64)


In [13]:
from ipywidgets import interact



def explore_3dimage(layer):
    plt.figure(figsize=(10, 5))
    channel = 3
    plt.imshow(image[0, :, :, layer]);
    plt.title('Explore Layers of Tumor prediction', fontsize=20)
    plt.axis('off')
    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_3dimage, layer=(0, 64 - 1));

interactive(children=(IntSlider(value=31, description='layer', max=63), Output()), _dom_classes=('widget-inter…

In [11]:
import pandas as pd 
c = pd.read_csv("/content/drive/My Drive/BRATS2020/MICCAI_BraTS2020_TrainingData/survival_info.csv")

In [12]:
import csv

def find_days_by_patient_number(input): 
    o = open('/content/drive/My Drive/BRATS2020/MICCAI_BraTS2020_TrainingData/survival_info.csv', 'r') 
    myData = csv.reader(o) 
    index = 0 
    
    for row in myData:

      if input in row[0]: 
        return row[2]
      else : index+=1
# ind = find_index('201')

In [ ]:
# print(ind)

317


In [ ]:
# with open('/content/drive/My Drive/BRATS2020/MICCAI_BraTS2020_TrainingData/survival_info.csv') as fd:
#     reader=csv.reader(fd)
#     interestingrows=[row for idx, row in enumerate(reader) if idx in (ind, ind)]

In [ ]:
# interestingrows

[['BraTS20_Training_032', '63.192', '488', 'NA']]

In [13]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. You'll need this later to add back to the main path.
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=1, strides=1, padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization( name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=f, strides=1, padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization( name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=1, strides=1, padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization( name=bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    ### END CODE HERE ###

    return X

In [14]:
def convolutional_block(X, f, filters, stage, block, s=2):
    """
    Implementation of the convolutional block as defined in Figure 4
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    # First component of main path
    X = Conv2D(F1, 1, strides=s, name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization( name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=f, strides=1, padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization( name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=1, strides=1, padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(name=bn_name_base + '2c')(X)

    # Shortcut path
    X_shortcut = Conv2D(F3, 1, strides=s, name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization( name=bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [15]:
def ResNet50(input_shape=(64, 64, 64)):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER
    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes
    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    res_X_input = Input(shape=input_shape)

    # Zero-Padding
    # X = ZeroPadding1D((3, 3))(X_input)

    # Stage 1
    res_X = Conv2D(64, 7, strides=1, name='conv1', kernel_initializer=glorot_uniform(seed=0))(res_X_input)
    res_X = BatchNormalization(name='bn_conv1')(res_X)
    res_X = Activation('relu')(res_X)
    res_X = MaxPooling2D(3, strides=2)(res_X)

    # Stage 2
    res_X = convolutional_block(res_X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    res_X = identity_block(res_X, 3, [64, 64, 256], stage=2, block='b')
    res_X = identity_block(res_X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3
    res_X = convolutional_block(res_X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    res_X = identity_block(res_X, 3, [128, 128, 512], stage=3, block='b')
    res_X = identity_block(res_X, 3, [128, 128, 512], stage=3, block='c')
    res_X = identity_block(res_X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    res_X = convolutional_block(res_X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    res_X = identity_block(res_X, 3, [256, 256, 1024], stage=4, block='b')
    res_X = identity_block(res_X, 3, [256, 256, 1024], stage=4, block='c')
    res_X = identity_block(res_X, 3, [256, 256, 1024], stage=4, block='d')
    res_X = identity_block(res_X, 3, [256, 256, 1024], stage=4, block='e')
    res_X = identity_block(res_X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    res_X = convolutional_block(res_X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    res_X = identity_block(res_X, 3, [512, 512, 2048], stage=5, block='b')
    res_X = identity_block(res_X, 3, [512, 512, 2048], stage=5, block='c')

    # Average Pooling
    res_X = AveragePooling2D(2, name='avg_pool')(res_X)

    ### END CODE HERE ###

    # output layer
    res_X = Flatten()(res_X)
    res_X = Dense(1024, activation='relu', name='fc0' , kernel_initializer=glorot_uniform(seed=0))(res_X)
    drp0 = Dropout(0.5)(res_X)
    res_X = Dense(256, activation='relu', name='fc1' , kernel_initializer=glorot_uniform(seed=0))(drp0)
    drp1 = Dropout(0.25)(res_X)
    res_X = Dense(64, activation='relu', name='fc2' , kernel_initializer=glorot_uniform(seed=0))(drp1)
    drp2 = Dropout(0.125)(res_X)
    res_X = Dense(1, activation='linear', name='fc3' , kernel_initializer=glorot_uniform(seed=0))(drp1)



    # Create model
    res_model = Model(inputs=res_X_input, outputs=res_X, name='ResNet50')

    return res_model

In [16]:
res_model = ResNet50(input_shape = (64, 64, 64))

In [17]:
res_model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 64)] 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 58, 58, 64)   200768      input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 58, 58, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, 58, 58, 64)   0           bn_conv1[0][0]                   
___________________________________________________________________________________________

In [54]:
def generator(X_path, y_path, batch_size=2):
    

   
    num_samples = len(X_path)
    while True: # Loop forever so the generator never terminates

      
      # Get index to start each batch: [0, batch_size, 2*batch_size, ..., max multiple of batch_size &lt;= num_samples]
      for offset in range(0, int(num_samples), batch_size):
            
          # Get the samples you'll use in this batch
          batch_samples = X_path[ (offset * batch_size) : ((offset+1) * batch_size) ]
 
            
             
              
              # Initialise X_train and y_train arrays for this batch
         

          for i in batch_samples:

            X_train = []
            y_train = []

            X_input = load_case(i)
            X = get_max(X_input, 64, 64, 64, 8, 10)
            
            rand_array = np.random.randint(10, size=3)
            
            for j in range(3):
              
              rand_num = rand_array[j]
              X_temp = np.reshape(X[rand_num, :, :, :], (64, 64, 64))
              


              patient_number = i[81:84]
              days = find_days_by_patient_number(patient_number)
              if days != None and 'ALIVE' not in days:
              
                X_train.append(X_temp)
                # for i in range(4):
                y_train.append(int(days))
                
                  
  
              # Make sure they're numpy arrays (as opposed to lists)

            X_train = np.array(X_train)
            y_train = np.array(y_train)

            




            if X_train.ndim == 4:             
              yield X_train, y_train
            else:
              pass

In [55]:

X_train_filenames = X_trainset_filenames[:309]
X_valid_filenames = X_trainset_filenames[309:]


y_train_path = '/content/drive/My Drive/BRATS2020/MICCAI_BraTS2020_TrainingData/survival_info.csv'

train_generator = generator(X_train_filenames, y_train_path, batch_size=3)
validation_generator = generator(X_valid_filenames, y_train_path, batch_size=3)

In [56]:

opt = Adam(lr=1e-4)


res_model.compile(optimizer=opt, loss='mean_absolute_percentage_error')

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

initial_learning_rate = 0.0001


def scheduler(epoch):
  
  ## decrease learning rate every 3 epochs to 0.1(latest_value) ##
  return float(initial_learning_rate * tf.math.exp(0.1 * (int(epoch/3))))
  


# callback
my_callbacks = [
        LearningRateScheduler(scheduler), 
        EarlyStopping(monitor='val_loss', patience=6),
        ModelCheckpoint(filepath='/content/drive/My Drive/BRATS2020/Survival_FlowerModel_update.{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', save_best_only=False)
]



#### steps per epoch should be x/batch*8 because model generates 8 subvolumes per training sample
# Fit model using generator
history = res_model.fit(train_generator, steps_per_epoch=309, epochs=50, validation_data=validation_generator, validation_steps=60, callbacks=my_callbacks)

Epoch 1/50
120/309 [==========>...................] - ETA: 21:57 - loss: 94.1249